# Libraries

In [1]:
import selenium 
import csv
import re
import pandas as pd
import ast
import numpy as np
import math

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options

import time

# Parser
- Convert original non-queryable names to queryable.

In [2]:
molecules_df = pd.read_csv('unmatched_valid_margaret.csv')
molecules_df = molecules_df[molecules_df['pubchem'].notna()]
pubnum_arr = molecules_df['pubchem'].to_list()
pubnum_arr = [int(x) for x in pubnum_arr]
pubnum_arr

# molecules_arr = molecules_df['preferred'].tolist()
# common_arr = molecules_df['common'].tolist()

# common_arr = [ast.literal_eval(s) for s in common_arr]

# pattern = re.compile(r'(\d)-(\d)')
# pattern2 = re.compile(r'_')
# pattern3 = re.compile(r'(\d)-([a-zA-Z])')

# # List to store updated molecule names
# updated_molecules_arr = []
# acc = 0

# for molecule in molecules_arr:
#     if isinstance(molecule, float) and (np.isnan(molecule) or math.isnan(molecule)):
#         if common_arr[acc] == []:
#             molecule_str = 'missing'
#         else:
#             molecule_str = common_arr[acc][0]
    
#     else:
#         molecule_str = molecule
    
#     updated_molecule = pattern.sub(r'\1,\2', molecule_str)
#     updated_molecule = pattern2.sub(r' ', updated_molecule)
#     updated_molecule = pattern3.sub(r'\1 \2', updated_molecule)
#     updated_molecules_arr.append(updated_molecule)
    
#     acc += 1

# updated_molecules_df = pd.DataFrame(updated_molecules_arr, columns=['updated_molecule'])
# updated_molecules_df

[157355,
 51049968,
 10472693,
 2159,
 9867812,
 51039094,
 11319217,
 70957463,
 52938427,
 10869981,
 25145656,
 6917864,
 44139752,
 71584930,
 10127622,
 86705695,
 57327016,
 118023034,
 66558664,
 69286,
 25267009,
 134436906,
 11518241,
 68748835,
 56944144,
 11962412,
 21302490,
 50248,
 56494,
 9822750,
 16722836,
 77843966,
 10130337,
 92786,
 44129647,
 67462786,
 56649450,
 11001318,
 44599207,
 456199,
 25141092,
 58557659,
 71661158,
 5311037,
 71481097,
 67171867,
 9941379,
 135565884,
 71602803,
 11610526,
 25185057,
 25151352,
 9948102,
 124081876,
 176077,
 11753673,
 5918,
 6324616,
 121225499,
 16124688,
 39524,
 11823650,
 9832912,
 6441636,
 134128281,
 25025298,
 3081431,
 89683805,
 31593,
 9863827,
 4021,
 44400135,
 44603531,
 450601,
 12068809,
 6029,
 221493,
 10430040,
 132999,
 11501341,
 56640146,
 6918638,
 45110509,
 9881504,
 11519070,
 15950376,
 5369209,
 449688,
 69046,
 65305,
 11434515,
 446284,
 132971,
 9825285,
 3086674,
 216210,
 6433083,
 135

In [3]:
len(pubnum_arr)

3649

In [4]:
links_arr = [f'https://pubchem.ncbi.nlm.nih.gov/compound/{str(x)}' for x in pubnum_arr]

# PubChem Query Result Scraper

In [14]:
# def get_pubchem_url(chemical):
#     return f'https://pubchem.ncbi.nlm.nih.gov/#query={chemical}'
def setup_webdriver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

# def get_best_match(molecule, driver):
#     url = "N/A"  # Default in case of failure
#     result_type = "Not Found"  # Default result type

#     driver.get(get_pubchem_url(molecule))
#     try:
#         WebDriverWait(driver, 6).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[data-action="featured-result-link"], a[data-action="result-link"]')))
#         elements = driver.find_elements(By.CSS_SELECTOR, 'a[data-action="featured-result-link"], a[data-action="result-link"]')
#         if elements:
#             element = elements[0]
#             url = element.get_attribute('href')
#             action = element.get_attribute('data-action')
#             if action == 'featured-result-link':
#                 result_type = 'FEATURED'
#             else:
#                 result_type = 'RELEVANT'
#     except (NoSuchElementException, TimeoutException):
#         pass  # URL remains "N/A" and result_type remains "Not Found" if no match is found or if there's a timeout

#     return molecule, url, result_type

# def get_best_matches(parsed_molecules):
#     best_matches = []
#     driver = setup_webdriver()  # Initialize the WebDriver once
#     try:
#         for parsed_molecule in parsed_molecules:
#             molecule, url, result_type = get_best_match(parsed_molecule, driver)
#             best_matches.append((molecule, url, result_type))
#     finally:
#         driver.quit()  # Make sure to quit the WebDriver

#     return best_matches

In [15]:
# best_matches = get_best_matches(updated_molecules_arr)

In [16]:
# df = pd.DataFrame(best_matches, columns=['Molecule', 'Link', 'Result Type'])

In [17]:
# df

In [18]:
# # df_merged = pd.concat([molecules_df, df], ignore_index=True, sort=False, axis=1)
# df_merged = df
# df_merged.columns = ['Parsed', 'Link', 'Result Type']
# # df_merged

In [19]:
# df_merged_csv = df_merged.to_csv('FINAL_MOLECULE_LINKS.csv', index = False) 

In [20]:
# links_arr = df_merged["Link"].tolist()
# links_arr

# PubChem CAS Scraper

In [21]:
def get_cas_numbers(link, driver):
    cas_numbers = {"CAS": "N/A", "Deprecated CAS": "N/A"}  # Default in case of failure

    try:
        driver.get(link)
        # Wait and locate the CAS number section
        try:
            cas_elements = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'section#CAS div.break-words')))
            cas_numbers["CAS"] = ', '.join([el.text.strip() for el in cas_elements])
        except (NoSuchElementException, TimeoutException):
            cas_numbers["CAS"] = "N/A"  # CAS number not found

        # Wait and locate the Deprecated CAS number section, if present
        try:
            deprecated_cas_elements = driver.find_elements(By.CSS_SELECTOR, 'section#Deprecated-CAS div.break-words')
            # deprecated_cas_elements = WebDriverWait(driver, 3).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, 'section#Deprecated-CAS div.break-words')))
            cas_numbers["Deprecated CAS"] = ', '.join([el.text.strip() for el in deprecated_cas_elements])
            
        except (NoSuchElementException, TimeoutException):
            cas_numbers["Deprecated CAS"] = "N/A"  # Deprecated CAS number not found
            
    except Exception as e:
        print(f"Error retrieving CAS numbers for {link}: {e}")

    return cas_numbers

def get_cas_numbers_concurrently(links):
    cas_matches = []
    driver = setup_webdriver()  # Initialize the WebDriver once

    try:
        for link in links:
            cas_matches.append(get_cas_numbers(link, driver))

    finally:
        driver.quit()  # Make sure to quit the WebDriver

    return cas_matches      

In [22]:
cas_numbers = get_cas_numbers_concurrently(links_arr)

In [23]:
df_cas = pd.DataFrame(cas_numbers, columns=['CAS', 'Deprecated CAS'])
df_cas.to_csv('task_2_unamtched_valid_pub_num_cas.csv', index = False)

# Combining Molecule Links Dataframe with CAS Dataframe

In [ ]:
final_df = pd.concat([df_merged, df_cas], axis=1)

In [ ]:
final_df.to_csv('task_2_unamtched_valid_cas.csv', index=False)

In [ ]:
final_df = pd.concat([molecules_df, final_df], axis=1)
final_df

In [ ]:
final_df.columns=['url', 'common', 'preferred', 'brand', 'inn', 'pubchem', 'who_atc', 'cas', 'originator', 'stereochemistry', 'formula', 'mw', 'investigational', 'us_market', 'us_market_year', 'us_prev_market', 'first_year_approval', 'fda_links', 'target_list', 'condition_list', 'patent_list', 'pubmed', 'pmid', 'mesh', 'Parsed Molecule', 'Link', 'Result Type', 'CAS', 'Deprecated CAS']
# final_df.to_csv('margaret_missing_cas._identified.csv', index=False)

# Flagging Compound Types

In [ ]:
final_df['Compound/Substance'] = final_df['Link'].apply(lambda x: 'COMPOUND' if 'compound' in str(x) else ('SUBSTANCE' if 'substance' in str(x) else 'N/A'))

In [ ]:
final_df

In [ ]:
# final_df['Original Molecule'] = final_df['Parsed Molecule']
# final_df = final_df[['Original Molecule', 'Parsed Molecule', 'Link', 'Result Type', 'CAS', 'Deprecated CAS', 'Compound/Substance']]
final_df

# Export Dataframe as CSV

In [ ]:
# final_df_csv = final_df.to_csv('pubchem_longer_wait.csv', index = False) 
final_df.to_csv('unmatched_valid_margaret_no_pubchem.csv', index=False)

In [ ]:
final_df

In [ ]:
molecules_df

In [ ]:
final_df = pd.concat([df_merged, df_cas], ignore_index=True, sort=False, axis=1)
final_df